# Solver de Sudoku usando Problemas de Satisfacción de Restricciones (CSP)

En este notebook, vamos a resolver puzles de Sudoku utilizando técnicas de Problemas de Satisfacción de Restricciones (CSP), como **Naked Single**, **Hidden Single** y algoritmos como **AC-3**, **MRV** (Valores Mínimos Restantes), y la **Heurística del Grado**.

## ¿Qué es un CSP?
Un **Problema de Satisfacción de Restricciones** es un problema matemático donde debemos asignar valores a un conjunto de variables de tal manera que se cumplan las restricciones entre ellas. El Sudoku es un ejemplo clásico de CSP, donde cada celda es una variable y las restricciones son las reglas del juego (cada fila, columna y cuadro 3x3 deben contener los dígitos del 1 al 9 sin repetirse).

## Importar librerías


In [ ]:
import os
import itertools as it
from collections import deque


## Función: `leer_tablero`

Esta función lee el tablero de Sudoku desde un archivo, asegurándose de que contenga exactamente 81 líneas (una por cada celda). Cada celda es o bien un valor conocido, o un conjunto de valores posibles.


In [ ]:
def leer_tablero(ruta_archivo):
    """Leer el archivo de texto y crear el diccionario del tablero."""
    tablero = {}
    with open(ruta_archivo, 'r') as archivo:
        lineas = archivo.readlines()
        if len(lineas) != 81:
            raise ValueError("El archivo debe contener exactamente 81 líneas.")
        filas = 'ABCDEFGHI'
        columnas = '123456789'
        for i, linea in enumerate(lineas):
            fila = filas[i // 9]
            columna = columnas[i % 9]
            casilla = fila + columna
            opciones = linea.strip()
            tablero[casilla] = set(opciones) if len(opciones) > 1 else {opciones}
    return tablero

def imprimir_tablero(tablero):
    """Imprimir el tablero de Sudoku."""
    filas = 'ABCDEFGHI'
    columnas = '123456789'
    for fila in filas:
        for columna in columnas:
            casilla = fila + columna
            print(''.join(tablero[casilla]) if len(tablero[casilla]) == 1 else '.', end=' ')
        print()


### Función: `unico_desnudo`

La técnica de **Naked Single** simplifica el tablero de Sudoku revisando si alguna celda contiene un único valor posible (un "single desnudo"). Si se encuentra tal celda, ese valor se elimina de las celdas vecinas en la misma fila, columna o caja.

Esto ayuda a reducir los valores posibles para las celdas vecinas y nos acerca más a la solución.


In [ ]:
def unico_desnudo(tablero, restricciones):
    """
    Aplica la técnica de Naked Single al tablero.

    Parámetros:
    tablero (dict): El tablero actual de Sudoku.
    restricciones (list): Una lista de restricciones (filas, columnas, cajas).

    Retorno:
    Ninguno: El tablero se modifica directamente.
    """
    for const in restricciones:
        for KeyVar in const:
            if len(tablero[KeyVar]) == 1:  # Si solo hay un valor posible
                for KeyXDelete in const:
                    if KeyXDelete != KeyVar:
                        # Eliminar ese valor de las demás celdas del mismo grupo
                        tablero[KeyXDelete].discard(next(iter(tablero[KeyVar])))


### Función: `unico_oculto`

Esta función implementa la técnica de **Hidden Single**. Recorre cada grupo de restricciones (fila, columna o caja) y verifica si un dígito solo puede colocarse en una celda específica. Si se encuentra dicha celda, asigna ese dígito a la celda.


In [ ]:
def unico_oculto(tablero, restricciones):
    """
    Aplica la técnica de Hidden Single al tablero de Sudoku.

    Parámetros:
    tablero (dict): El tablero actual de Sudoku.
    restricciones (list): Una lista de restricciones (filas, columnas, cajas).

    Retorno:
    Ninguno: El tablero se modifica directamente.
    """
    for const in restricciones:
        for digito in '123456789':
            contador = 0
            ultima_clave = None
            for KeyVar in const:
                # Verificar si el dígito es un valor posible para esta celda
                if digito in tablero[KeyVar]:
                    contador += 1
                    ultima_clave = KeyVar
            # Si el dígito solo puede estar en una celda, debe colocarse allí
            if contador == 1:
                tablero[ultima_clave] = {digito}


### Algoritmo: `AC-3` (Arc Consistency 3)

**AC-3** es un algoritmo fundamental usado para aplicar consistencia entre variables en CSP. En Sudoku, ayuda reduciendo los valores posibles para cada celda revisando pares de celdas vecinas. Si una celda tiene un valor que contradice a un vecino, ese valor se elimina.

Esto es útil especialmente para eliminar valores imposibles desde el principio en el proceso de búsqueda.


In [ ]:
def ac3(tablero, restricciones):
    """
    Implementa el algoritmo AC-3 para reducir el dominio de variables en el tablero de Sudoku.

    Parámetros:
    tablero (dict): El tablero actual de Sudoku.
    restricciones (list): Una lista de restricciones (filas, columnas, cajas).

    Retorno:
    bool: Devuelve True si el tablero es consistente en arcos, de lo contrario False.
    """
    queue = deque([(Xi, Xj) for const in restricciones for Xi in const for Xj in const if Xi != Xj])

    while queue:
        Xi, Xj = queue.popleft()
        if revisar_consistencia_arco(Xi, Xj, tablero):
            if len(tablero[Xi]) == 0:
                return False
            for const in restricciones:
                if Xi in const:
                    for Xk in const:
                        if Xk != Xi and Xk != Xj:
                            queue.append((Xk, Xi))
    return True

def revisar_consistencia_arco(Xi, Xj, tablero):
    """Revisar si el dominio de Xi puede reducirse eliminando valores inconsistentes con Xj."""
    eliminado = False
    for x in tablero[Xi].copy():
        if not any(x != y for y in tablero[Xj]):
            tablero[Xi].discard(x)
            eliminado = True
    return eliminado


### Función: `mrv`

La heurística **MRV** (Valores Mínimos Restantes) se usa para seleccionar la variable (celda del Sudoku) con el menor número de valores posibles restantes. Esta heurística está basada en la idea de que la variable más restringida debe resolverse primero, ya que limita futuras elecciones y simplifica el problema.

Para el Sudoku, esto se traduce en elegir la celda con el menor número de dígitos posibles para asignar.


In [ ]:
def mrv(tablero):
    """
    Heurística MRV (Valores Mínimos Restantes): Selecciona la celda con el menor número de valores posibles.

    Parámetros:
    tablero (dict): El tablero actual de Sudoku.

    Retorno:
    str: La celda (clave) con los valores mínimos restantes.
    """
    # Devuelve la celda con el menor número de valores posibles, ignorando las celdas ya resueltas (con 1 valor)
    return min((v for v in tablero if len(tablero[v]) > 1), key=lambda x: len(tablero[x]), default=None)


### Función: `heuristica_grado`

La **Heurística de Grado** se utiliza cuando varias variables tienen el mismo número de valores restantes. Desempata seleccionando la variable que participa en el mayor número de restricciones (es decir, la celda que afecta a más celdas vecinas).

Esto ayuda a tomar la decisión más restrictiva, reduciendo aún más el espacio de búsqueda.


In [ ]:
def heuristica_grado(tablero, restricciones):
    """
    Heurística de Grado: Selecciona la celda que participa en el mayor número de restricciones.

    Parámetros:
    tablero (dict): El tablero actual de Sudoku.
    restricciones (list): La lista de restricciones (filas, columnas, cajas).

    Retorno:
    str: La celda (clave) que participa en el mayor número de restricciones.
    """
    return max((v for v in tablero if len(tablero[v]) > 1), key=lambda x: sum(1 for c in restricciones if x in c), default=None)


### Función: `resolver`

La función `resolver` utiliza **backtracking** para encontrar una solución al puzle de Sudoku. Emplea tanto la heurística MRV como la Heurística de Grado para guiar el proceso de búsqueda, reduciendo el espacio de búsqueda y haciendo que el proceso sea más eficiente.


In [ ]:
def es_tablero_completo(tablero):
    """Verifica si el tablero está completo."""
    return all(len(tablero[var]) == 1 for var in tablero)

def resolver(tablero, restricciones, verbose=False):
    """
    Resuelve el puzle de Sudoku usando backtracking, MRV, y Heurística de Grado.

    Parámetros:
    tablero (dict): El tablero actual de Sudoku.
    restricciones (list): La lista de restricciones (filas, columnas, cajas).
    verbose (bool): Si es True, imprime los pasos tomados por el solucionador.

    Retorno:
    dict: El tablero resuelto, o None si no se encuentra solución.
    """
    # Verifica si el tablero está completamente resuelto
    if es_tablero_completo(tablero):
        return tablero

    # Selecciona la variable a asignar un valor usando MRV, o Heurística de Grado para desempatar
    var = mrv(tablero) or heuristica_grado(tablero, restricciones)

    if not var:
        return None

    # Intenta asignar cada valor posible a la variable seleccionada
    for valor in tablero[var].copy():
        # Crea una copia del tablero para probar la asignación
        tablero_copia = {v: tablero[v].copy() for v in tablero}
        tablero_copia[var] = {valor}

        # Si la asignación es válida (AC3 es consistente), continúa buscando
        if ac3(tablero_copia, restricciones):
            if verbose:
                print(f"Asignando {valor} a la variable {var}")
            solucion = resolver(tablero_copia, restricciones, verbose)
            if solucion:
                return solucion

    return None


### Ejemplo: Resolviendo un Puzle de Sudoku

Probemos ahora el solucionador CSP completo en un puzle difícil de Sudoku.


In [ ]:
def generar_restricciones():
    """Genera las restricciones para el Sudoku."""
    filas = 'ABCDEFGHI'
    columnas = '123456789'

    def agrupar(cajas):
        return [tuple(caja) for caja in cajas]

    # Filas
    filas_grupo = agrupar([[fila + columna for columna in columnas] for fila in filas])
    # Columnas
    columnas_grupo = agrupar([[fila + columna for fila in filas] for columna in columnas])
    # Cajas 3x3
    cajas_grupo = agrupar([[fila + columna for fila in filas[i:i+3] for columna in columnas[j:j+3]] for i in range(0, 9, 3) for j in range(0, 9, 3)])

    return filas_grupo + columnas_grupo + cajas_grupo

def main():
    ruta_default = r'Board_impossible_SD9BJKIA.txt'
    ruta_archivo = input("Ingrese la ruta del archivo de Sudoku (o presione Enter para usar la ruta por defecto): ")
    if not ruta_archivo:
        ruta_archivo = ruta_default

    if not os.path.exists(ruta_archivo):
        print(f"El archivo {ruta_archivo} no existe.")
        return

    tablero = leer_tablero(ruta_archivo)
    restricciones = generar_restricciones()

    print("Tablero inicial:")
    imprimir_tablero(tablero)

    solucion = resolver(tablero, restricciones, verbose=True)

    if solucion:
        print("\n¡Solución encontrada!")
        imprimir_tablero(solucion)
    else:
        print("\nNo se encontró solución.")

if __name__ == "__main__":
    main()


Ingrese la ruta del archivo de Sudoku (o presione Enter para usar la ruta por defecto): 
Tablero inicial:
. 6 . . 5 . . . 4 
. . 9 . . . . . . 
7 1 5 . . . . . . 
. . 3 . 1 . . . 5 
. . . 7 . 3 2 . . 
8 . 1 . 4 5 . 9 . 
. . 7 . . . 9 . . 
2 . 8 . . . 5 4 . 
6 . . 9 2 . 7 . . 
Asignando 3 a la variable A1
Asignando 1 a la variable A4
Asignando 6 a la variable C7
Asignando 3 a la variable B9
Asignando 2 a la variable B9
Asignando 8 a la variable A4
Asignando 6 a la variable B7
Asignando 3 a la variable B5

¡Solución encontrada!
3 6 2 8 5 9 1 7 4 
4 8 9 1 3 7 6 5 2 
7 1 5 4 6 2 8 3 9 
9 7 3 2 1 8 4 6 5 
5 4 6 7 9 3 2 1 8 
8 2 1 6 4 5 3 9 7 
1 3 7 5 8 4 9 2 6 
2 9 8 3 7 6 5 4 1 
6 5 4 9 2 1 7 8 3 
